# Set Up

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import re

from scipy.stats import skew, chi2_contingency
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.impute import SimpleImputer

from wordsegment import load, segment
load()
import nltk
from nltk.corpus import words, brown
import spacy

nltk.download('words')
nltk.download('brown')

# Get set of English words
english_words = set(words.words())

pd.set_option('display.max_columns', 15)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\ProgramData\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\ProgramData\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "c:\ProgramData\anaconda3\Lib\site-pack

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

In [ ]:
# Read the main file
main_df = pd.read_csv('dga_data_small.csv')
main_df.head()

In [ ]:
main_df['tld'] = main_df['host'].str.split('.', n=1).str[1]

main_df['cctld'] = main_df['tld'].str.split('.', n=1).str[1].fillna('None') # Code country top level domain

main_df

main_df = main_df.rename(columns = {'domain': 'subdomain'})

In [ ]:
def is_ascii_domain(subdomain):
    ascii_pattern = re.compile(r'^[a-zA-Z0-9.-]+$')
    return int(bool(ascii_pattern.match(subdomain)))
    
main_df['ascii'] = main_df['subdomain'].apply(is_ascii_domain)

# Domain length
main_df['subdomain_len'] = main_df['subdomain'].str.len()
main_df['host_len'] = main_df['host'].str.len()

In [ ]:
main_df

In [ ]:
# Vowel count
main_df['subdomain_vowel_count'] = main_df['subdomain'].str.lower().str.count(r'[aeoiu]')

# Consonant count
main_df['subdomain_consonant_count'] = main_df['subdomain_len'] - main_df['subdomain_vowel_count']

# Has Numeric - boolean to int type
main_df['has_num'] = main_df['subdomain'].str.contains(r'\d').astype(int)

In [ ]:
def legitToNumber(isdga):
    if isdga == 'legit':
        return 1
    else: return 0

main_df['isDGA_N'] = main_df['isDGA'].apply(legitToNumber)

main_df['digitCount'] = main_df['subdomain'].str.count(r'\d')

main_df.head()

In [ ]:
OH_encoder = OneHotEncoder(sparse_output=False)

In [ ]:
encoded = OH_encoder.fit_transform(np.array(main_df['subclass']).reshape(-1, 1))
encoded_features = pd.DataFrame(encoded, columns=OH_encoder.get_feature_names_out(['subclass']))

main_df = main_df.join(encoded_features)

main_df.head()

In [ ]:
domain_cross_check = pd.crosstab(main_df['subclass'],main_df['isDGA_N'])
domain_cross_check

In [ ]:
# Check if subdomain starting with digit
main_df['startW/Digit'] = main_df['subdomain'].str.match(r'^\d').astype(int)

# EDA

In [ ]:
main_df

In [ ]:
for col in main_df.columns:
    print(f'Unique values for {col}: {main_df[col].unique()}\n')

In [ ]:
legit_df = main_df[(main_df['isDGA']=='legit')].reset_index().drop(columns='index')

legit_df

In [ ]:
legit_df['tld'].unique()

In [ ]:
dga_df = main_df[(main_df['isDGA']!='legit')].reset_index().drop(columns='index')

dga_df

In [ ]:
legit_df['subdomain_len'].mean()

In [ ]:
dga_df['subdomain_len'].mean()

# Sample

In [ ]:
# def is_composed_of_english_words(subdomain):
#     possible_words = []
#     found_words = []
    
#     for i in range(len(subdomain)-1):
#         for j in range(i+1, len(subdomain)-1):
#             possible_words.append(subdomain[i:j])

#     for word in possible_words:
#         if len(word) >= 2 and word in english_words:
#             found_words.append(word)
    
#     return found_words

# sample = 'teacherspayteachers'
# print(is_composed_of_english_words(sample))

### Word Segment lib

In [ ]:
legit_df_subdomain = legit_df[['subdomain']].copy()

In [ ]:
legit_df_subdomain['extraction'] = legit_df_subdomain['subdomain'].apply(segment)

In [ ]:
legit_df_subdomain['extraction']

In [ ]:
from textblob import TextBlob
word = 'internet'
TextBlob(word).correct() == word

# print(TextBlob(word).correct())

In [ ]:
def count_true_textblob(list_words):
    valid = []
    for i in list_words:
        if TextBlob(i).correct() == i:
            valid.append(i)

    return valid

def count_true_nltk(list_words):
    valid = []
    for i in list_words:
        if i in english_words:
            valid.append(i)

    return valid

legit_df_subdomain['validation_textblob'] = legit_df_subdomain['extraction'].apply(count_true_textblob)
legit_df_subdomain['validation_nltk'] = legit_df_subdomain['extraction'].apply(count_true_nltk)

In [ ]:
legit_df_subdomain

### Brown

In [ ]:
legit_df_subdomain.iloc[291]

In [ ]:
word_freq = nltk.FreqDist(brown.words())

In [ ]:
word = "ache"
frequency = word_freq[word.lower()]
print(f"The word '{word}' appears {frequency} times in the Brown corpus.")

In [ ]:
print(brown.categories())

In [ ]:
type(brown)

In [ ]:
brown.words(categories='news')

In [ ]:
brown.sents()[5]